# Convolutional Neural Network

In [1]:
import pandas as pd
import numpy as np
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.models import Model, Input, Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, SpatialDropout1D, Activation
from keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization
from keras.optimizers import Adam

Using TensorFlow backend.


In [16]:
in_folder='data/3-processed_data'
out_folder='models'

In [3]:
X_train=pd.read_csv(in_folder+'/X_train.csv')
Y_train=pd.read_csv(in_folder+'/Y_train.csv')
X_test=pd.read_csv(in_folder+'/X_test.csv')
Y_test=pd.read_csv(in_folder+'/Y_test.csv')

In [4]:
# number of unique words we want to use (or: number of rows in incoming embedding vector)
max_features = 20000 

# max number of words in a comment to use (or: number of columns in incoming embedding vector)
max_len = 200 

# dimension of the embedding variable (or: number of rows in output of embedding vector)
embedding_dims = 128

In [8]:
# instantiate CNN model
cnn_model = Sequential()

# add embedding layer 
cnn_model.add(Embedding(input_dim=max_features, input_length=max_len,
                        output_dim=embedding_dims))
 
# set the dropout layer to drop out 50% of the nodes
cnn_model.add(SpatialDropout1D(0.5))

# add convolutional layer that has ...
# ... 100 filters with a kernel size of 4 so that each convolution will consider a window of 4 word embeddings
cnn_model.add(Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))

# add normalization layer
cnn_model.add(BatchNormalization())

# add pooling layer 
cnn_model.add(GlobalMaxPool1D())

# set the dropout layer to drop out 50% of the nodes
cnn_model.add(Dropout(0.5))

# add dense layer to produce an output dimension of 50 and using relu activation
cnn_model.add(Dense(50, activation='relu'))

# finally add a dense layer
cnn_model.add(Dense(7, activation='sigmoid'))

In [9]:
cnn_model.compile(loss='binary_crossentropy',
                  optimizer=Adam(0.01),
                  metrics=['accuracy'])

cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 128)          2560000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 200, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 200, 100)          51300     
_________________________________________________________________
batch_normalization_2 (Batch (None, 200, 100)          400       
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 100)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
__________

In [10]:
cnn_hist = cnn_model.fit(X_train, Y_train, batch_size=256, 
                         epochs=4, validation_split=0.2)

Train on 127656 samples, validate on 31915 samples
Epoch 1/4
127656/127656 [==============================] - 297s 2ms/step - loss: 0.0984 - acc: 0.9686 - val_loss: 0.0663 - val_acc: 0.9769
Epoch 2/4
127656/127656 [==============================] - 323s 3ms/step - loss: 0.0637 - acc: 0.9773 - val_loss: 0.0629 - val_acc: 0.9777
Epoch 3/4
127656/127656 [==============================] - 329s 3ms/step - loss: 0.0569 - acc: 0.9792 - val_loss: 0.0639 - val_acc: 0.9778
Epoch 4/4
127656/127656 [==============================] - 315s 2ms/step - loss: 0.0529 - acc: 0.9803 - val_loss: 0.0657 - val_acc: 0.9775


In [11]:
cnn_test_loss, cnn_test_acc = cnn_model.evaluate(X_test, Y_test, batch_size=32)
print('Test Loss:    ', cnn_test_loss)
print('Test Accuracy:', cnn_test_acc)

63978/63978 [==============================] - 51s 796us/step
Test Loss:     0.08784714866866392
Test Accuracy: 0.9653452397652403


## Saving the model

In [17]:
#Saving Model
cnn_model.save(out_folder+'/cnn_model.h5')

In [18]:
#Saving Performance
models_performance=pd.read_csv('models/models_performance.csv')
stats=['Convolutional Model', cnn_test_loss,cnn_test_acc]
models_performance.loc[len(models_performance),:]=stats
models_performance.to_csv(out_folder+'/models_performance.csv',index=False)